# RAG系统完整流程演示

本 Notebook 演示了 RAG（检索增强生成）系统的完整使用流程，从文档加载到最终生成回答。

## 流程概览

```
文档加载 → 文档分块 → 向量嵌入 → 向量索引 → 相似度搜索 → 结果过滤 → 文本生成
```


## 1. 环境准备

首先导入必要的库和设置 API 基础地址。


In [ ]:
import requests
import json
from typing import Dict, List
import time

# API 基础地址
BASE_URL = "http://localhost:8000/api"

def print_result(title: str, data: Dict):
    """格式化打印结果"""
    print(f"\n{'='*50}")
    print(f"{title}")
    print(f"{'='*50}")
    print(json.dumps(data, indent=2, ensure_ascii=False))


## 2. 文档加载 (Loading)

上传文档并获取 `file_id`。支持 PDF、DOCX、TXT 等格式。


In [ ]:
# 方式1：上传文件
# 方式2：指定文件路径
file_path = "../backend/data/upload/大模型RAG进阶实战营 —开营导语.pdf"

with open(file_path, "rb") as f:
    files = {"file": f}
    data = {"method": "pymupdf"}  # 可选: pymupdf, pypdf, unstructured
    response = requests.post(f"{BASE_URL}/loading/upload", files=files, data=data)
    
loading_result = response.json()
file_id = loading_result["file_id"]

print_result("文档加载结果", loading_result)
print(f"\n✅ 获取到 file_id: {file_id}")


## 3. 文档分块 (Chunking)

将文档切分成小的文本块，便于后续向量化处理。


In [ ]:
chunking_request = {
    "file_id": file_id,
    "chunking_strategy": "by_size",  # 可选: by_size, by_sentence, by_paragraph
    "chunk_size": 1000,  # 每个块的大小（字符数）
    "overlap": 200  # 块之间的重叠大小
}

response = requests.post(f"{BASE_URL}/chunking/chunk", json=chunking_request)
chunking_result = response.json()
chunk_id = chunking_result["chunk_id"]

print_result("文档分块结果", chunking_result)
print(f"\n✅ 获取到 chunk_id: {chunk_id}")
print(f"📊 总块数: {chunking_result.get('total_chunks', 0)}")


## 4. 向量嵌入 (Embedding)

将文本块转换为数值向量，使用本地 embedding 模型。


In [ ]:
embedding_request = {
    "chunk_id": chunk_id
}

response = requests.post(f"{BASE_URL}/embedding/embed", json=embedding_request)
embedding_result = response.json()
embedding_id = embedding_result["embedding_id"]

print_result("向量嵌入结果", {
    "embedding_id": embedding_id,
    "model": embedding_result.get("model"),
    "embedding_dim": embedding_result.get("embedding_dim"),
    "total_chunks": embedding_result.get("total_chunks")
})
print(f"\n✅ 获取到 embedding_id: {embedding_id}")
print(f"📐 向量维度: {embedding_result.get('embedding_dim', 'N/A')}")


## 5. 向量索引 (Indexing)

将向量存储到 ChromaDB 中，建立可搜索的索引。


In [ ]:
collection_name = "test"

indexing_request = {
    "embedding_id": embedding_id,
    "collection_name": collection_name
}

response = requests.post(f"{BASE_URL}/indexing/index", json=indexing_request)
indexing_result = response.json()
index_id = indexing_result["index_id"]

print_result("向量索引结果", indexing_result)
print(f"\n✅ 获取到 index_id: {index_id}")
print(f"📁 集合名称: {collection_name}")
print(f"📊 文档数: {indexing_result.get('total_documents', 0)}")


## 6. 相似度搜索 (Similarity Search)

使用自然语言查询，在向量数据库中搜索相关文档。


In [ ]:
query_text = "为什么要学习RAG？"

search_request = {
    "collection_name": collection_name,
    "query_text": query_text,
    "n_results": 5  # 返回最相似的5条结果
}

response = requests.post(f"{BASE_URL}/indexing/search", json=search_request)
search_result = response.json()

print_result("相似度搜索结果", {
    "query_text": query_text,
    "results_count": search_result.get("results_count"),
    "collection_name": search_result.get("collection_name")
})

print("\n📋 搜索结果详情:")
for i, item in enumerate(search_result.get("results", [])[:3], 1):
    print(f"\n结果 {i}:")
    print(f"  ID: {item.get('id')}")
    print(f"  距离: {item.get('distance', 0):.4f}")
    print(f"  内容: {item.get('document', '')[:200]}...")


## 7. 结果过滤 (Filtering) - 可选

对搜索结果进行过滤和排序，提高后续生成的质量。


In [ ]:
filter_request = {
    "search_results": search_result,
    "min_score": 0.5,  # 最小相似度分数（0.0-1.0）
    "max_score": 1.0,  # 最大相似度分数
    "sort_order": "desc"  # 降序排列（相似度高的在前）
}

response = requests.post(f"{BASE_URL}/filtering/filter", json=filter_request)
filter_result = response.json()
filter_id = filter_result.get("filter_id")

print_result("结果过滤", {
    "filter_id": filter_id,
    "原始结果数": filter_result.get("original_count"),
    "过滤后数量": filter_result.get("filtered_count")
})
print(f"\n✅ 获取到 filter_id: {filter_id}")


## 8. 文本生成 (Generation)

基于检索到的相关文档，生成回答。这是 RAG 系统的最终输出。


In [ ]:
# 方式1：使用过滤后的结果
generation_request = {
    "query": query_text,
    "filter_id": filter_id  # 使用过滤后的结果
}

# 方式2：直接使用搜索结果（如果跳过过滤步骤）
# generation_request = {
#     "query": query_text,
#     "search_results": search_result  # 直接使用搜索结果
# }

response = requests.post(f"{BASE_URL}/generation/generate", json=generation_request)
generation_result = response.json()
generation_id = generation_result.get("generation_id")

print_result("文本生成结果", {
    "generation_id": generation_id,
    "query": query_text,
    "status": generation_result.get("status")
})

print("\n🤖 生成的回答:")
print("-" * 50)
print(generation_result.get("answer", ""))
print("-" * 50)


## 9. 完整流程总结

以下是整个流程中生成的所有 ID，用于追踪和调试：


In [ ]:
print("\n📝 RAG流程ID追踪:")
print(f"  1. 文档加载    → file_id:      {file_id}")
print(f"  2. 文档分块    → chunk_id:     {chunk_id}")
print(f"  3. 向量嵌入    → embedding_id: {embedding_id}")
print(f"  4. 向量索引    → index_id:     {index_id}")
print(f"  5. 结果过滤    → filter_id:    {filter_id}")
print(f"  6. 文本生成    → generation_id: {generation_id}")

print("\n✅ RAG系统完整流程执行成功！")


## 10. 查看历史记录

可以查看各个模块的历史处理记录。


In [ ]:
# 查看索引历史
response = requests.get(f"{BASE_URL}/indexing/history")
indexing_history = response.json()
print(f"\n📚 索引历史记录数: {indexing_history.get('total', 0)}")

# 查看生成历史
response = requests.get(f"{BASE_URL}/generation/history")
generation_history = response.json()
print(f"📚 生成历史记录数: {generation_history.get('total', 0)}")


## 注意事项

1. **确保后端服务运行**：在运行此 notebook 前，确保后端服务已启动（`python backend/main.py`）
2. **文件路径**：请将文档加载部分的文件路径替换为实际文件路径
3. **API 地址**：如果后端运行在不同端口，请修改 `BASE_URL`
4. **依赖安装**：确保已安装所有必要的 Python 包（见 `requirements.txt`）
5. **模型下载**：首次使用 embedding 模型时，会自动从 Hugging Face 下载

## 更多信息

详细的功能说明请参考：`RAG系统功能模块关系说明.md`
